<h1 align="center">MSIN0114: Business Analytics Consulting Project</h1>
<h2 align="center">S2R Analytics</h2>

# Table of Contents

* [Part 1](#part1): Data extraction

* [Part 2](#part2): Data transformation
    * [2.1](#2_1): Projects
    * [2.2](#2_2): Transactions
    * [2.3](#2_3): Stages
    * [2.4](#2_4): Data health
    * [2.5](#2_5): Clients 
    * [2.6](#2_6): Staff
    * [2.7](#2_7): Clean-up    
 <br />
 
* [Part 3](#part3): Data loading
    * [3.1](#3_1): Database design and storage
    * [3.2](#3_2): Conversion to flat file

## Notebook Setup

In [1]:
# Essentials
import pandas as pd
from pandas import Series, DataFrame
from pandas.api.types import CategoricalDtype
pd.options.display.max_columns = None
import numpy as np; np.random.seed(2022)
import random
import sqlite3
import pyodbc

# Image creation and display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from matplotlib import pyplot
import plotly.express as px
import plotly.graph_objects as go

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.base import clone
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier

# Other
import itertools as it
import io
import os
os.sys.path
import sys
import glob
import concurrent.futures
from __future__ import print_function
import binascii
import struct
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster
import datetime, time
import functools, operator
from datetime import datetime

## Part 1: <a class="anchor" id="part1"></a> Data extraction

Main data is gathered from API scripts granted by Jonny, the CEO of S2R Analytics. Only minor tweaks were needed to change the paths for the needed data tab;es from Total Synergy. Check the code for one of the tables transformed below in ``` ETL\data_extraction.py ```.

## Part 2: <a class="anchor" id="part2"></a> Data transformation

### 2.1 <a class="anchor" id="2_1"></a> Projects

This section resulted in *aec.projects* table in RDS for Client X. Noteworthy, more features are added beyond this section, using other data tables.

Step 1: Create a list of projects to drop.

In [2]:
# Read all projects from Synergy API
all_projects = pd.read_csv('csv-files/wga_synergy_incremental_projects.csv')
all_projects = all_projects[['Project ID', 'Project Number', 'Project Name', 'Is Office Project', 'Is Billable', 'Project Status']]

# Projects to keep: external (i.e. client only)
external_projects = all_projects[(all_projects['Is Office Project'] != 'Yes')]
external_projects = external_projects[(external_projects['Is Billable'] != 'No')]
external_ids = external_projects['Project ID'].tolist()

# Projects to keep: status-based
successful_projects = external_projects[external_projects['Project Status'].isin(['Complete', 'Active', 'Pending Invoice']) == True]
valid_ids = successful_projects['Project ID'].tolist()

# See how many unique projects we shold have
print('We should have ' + str(len(valid_ids)) + ' projects in total.')

We should have 9755 projects in total.


In [ ]:
print('Hence, ' + str(len(all_projects) - len(valid_ids)) + ' projects were dropped for being invalid.')

Hence, 5570 projects were dropped for being invalid.


Step 2: Cleaning data from Synergy API.

In [ ]:
# Load only valid projects
api_projects = pd.read_csv('csv-files/wga_synergy_incremental_projects.csv', parse_dates=['Start Date (Project)', 'End Date (Project)'], dayfirst=True)
api_projects = (api_projects[api_projects['Project ID'].isin(valid_ids)])


# Drop unnecesary columns
api_projects.drop(columns = ['Unnamed: 0', 'Primary Contact Name', 'Status Name', 'Organisation ID',
                             'customFields', 'Address Line 1', 'Address Line 2', 'Project Type ID',
                             'Primary Contact', 'Primary Contact ID', 'Project Scope', 'Address Postal Code',
                             'Address State', 'Address Town', 'Address Google', 'Client Reference Number',
                             'Address State Postal Code Country', 'Address Single Line', 'Project Type Code',
                             'External Name', 'Address Longitude', 'Address Latitude', 'Project Manager', 
                             'Project Forecast Value', 'Created Date', 'Updated Date'], inplace = True)


# Convert columns for unified style
api_projects.rename(columns = {'Invoices':'Number of Invoices', 'Project Net Residual (Neg as Zero)':'Project Net Residual',
                              'Start Date (Project)': 'Project Start Date', 'End Date (Project)': 'Project End Date',
                              'Address Country':'Country', 'Project Type': 'Sector'}, inplace = True)
api_projects['Country'].replace(['AUSTRALIA', 'AUS', 'Autralia', 'NZ', 'new zealand', 'PNG', 'samoa', 'SAMOA', 'TONGA', 'SA', 'CHINA'],
                                ['Australia', 'Australia', 'Australia', 'New Zealand', 'New Zealand', 'Papua New Guinea', 'Samoa', 'Samoa', 'Tonga', 'Saudi Arabia', 'China'],inplace=True)


# Generalise minority observations into bigger groups
api_projects['Sector'].mask(api_projects['Sector'] == 'Commercial', 'Commercial & Retail Buildings', inplace=True)
api_projects['Sector'].mask(api_projects['Sector'] == 'Residential', 'Civic & Education Buildings', inplace=True)
api_projects['Default Rate Group'].mask(api_projects['Default Rate Group'] != 'Standard', 'Non-standard', inplace=True)


# Adding 'Due Date' and'Project Director' columns
custom_fields = pd.read_csv('csv-files/wga_synergy_incremental_projects_custom_fields.csv')
custom_fields = custom_fields[['PROPOSAL - Due Date', 'PROSPECT - Project Director', 'Project ID']].copy()
custom_fields.rename(columns = {'PROSPECT - Project Director':'Project Director', 'PROPOSAL - Due Date': 'Due Date'}, inplace = True)
custom_fields['Project Director'] = custom_fields['Project Director'].str.rstrip(' (MPD)')
custom_fields['Due Date'] = pd.to_datetime(custom_fields['Due Date'])
api_projects = pd.merge(api_projects, custom_fields,  how='left', on='Project ID')


# Rearrange column names for easier interpretation
api_projects = api_projects[['Project ID', 'Country',
                             'Project Status', 'Sector',
                             'Project Director', 'Manager ID', 'Office',
                             'Project Start Date', 'Project End Date', 'Due Date',
                             'Default Rate Group']]

len(api_projects)

9755

Step 3: Cleaning transformed PowerBI data from S2R Analytics.

In [ ]:
# Read the pre-transformed data from PowerBI
pbi_projects = pd.read_csv('csv-files/wga_power_bi_projects.csv', encoding = 'ISO-8859-1', low_memory=False)
pbi_projects = pbi_projects[['Project ID', 'Project Size Sort Order', 'Is Multi Discipline Project','Is First Client Project']]

# Load only valid projects
pbi_projects = (pbi_projects[pbi_projects['Project ID'].isin(valid_ids)])

# Convert columns for unified style
pbi_projects['Is Multi Discipline Project'].replace(['No', 'Yes'],[False, True],inplace=True)
pbi_projects['Is First Client Project'].replace(['No', 'Yes'],[False, True],inplace=True)

# Convert the object column to boolean
pbi_projects['Is Multi Discipline Project'] = pbi_projects['Is Multi Discipline Project'].astype(bool)
pbi_projects['Is Multi Discipline Project'] = pbi_projects['Is Multi Discipline Project'].astype(int)
pbi_projects['Is First Client Project'] = pbi_projects['Is First Client Project'].astype(int)

len(pbi_projects)

9754

Step 4: Merge the two 'Projects' tables together.

In [ ]:
# Merge the projects table from API and preprocesed Power BI table
projects = pd.merge(api_projects, pbi_projects,  how='left', left_on='Project ID', right_on='Project ID')
projects.columns = projects.columns.str.replace(' ', '_')
projects.head(1)

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project
0,367704,Australia,Complete,Ports & Marine,Mark Gilbert,7612852,Whyalla,2015-07-01,2015-08-01,NaT,Standard,4.0,0.0,1.0


In [ ]:
projects.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9755 entries, 0 to 9754
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Project_ID                   9755 non-null   int64         
 1   Country                      9640 non-null   object        
 2   Project_Status               9755 non-null   object        
 3   Sector                       9755 non-null   object        
 4   Project_Director             9754 non-null   object        
 5   Manager_ID                   9755 non-null   int64         
 6   Office                       9755 non-null   object        
 7   Project_Start_Date           4061 non-null   datetime64[ns]
 8   Project_End_Date             4037 non-null   datetime64[ns]
 9   Due_Date                     1977 non-null   datetime64[ns]
 10  Default_Rate_Group           9755 non-null   object        
 11  Project_Size_Sort_Order      9754 non-null 

In [ ]:
# URL: https://stackoverflow.com/questions/21295334/find-length-of-longest-string-in-pandas-dataframe-column
int(projects['Country'].astype(bytes).str.len().max())

20

In [ ]:
int(projects['Office'].astype(bytes).str.len().max())

38

In [ ]:
int(projects['Sector'].astype(bytes).str.len().max())

41

In [ ]:
int(projects['Default_Rate_Group'].astype(bytes).str.len().max())

12

**5 features to engineer:**
* Project_Duration_Weeks
* Delivered_on_Time
* Fully_In_Lockdown
* Partially_In_Lockdown
* Suffered_Data_Loss (projects that started after July 2018 did not suffer from data loss, projects that ended before July 2018 did not suffer from data loss)

The features are engineered after dealing with transactions table to fill in blanks of missing start and end dates of projects.

### 2.2 <a class="anchor" id="2_2"></a> Transactions

This section resulted in *aec.transactions* table in RDS for Client X.

In [ ]:
# Read only valid projects' transactions from Synergy API.
transactions = pd.read_csv('csv-files/wga_sql_transactions.csv', parse_dates=['date'], dayfirst=True, low_memory=False)
transactions = (transactions[transactions['projectId'].isin(valid_ids)])

transactions = transactions[['id', 'projectId', 'stageId', 'transactionTypeId',
                             'rateType', 'status','units','valueTotal',
                             'invoiceValueTotal','actualCostTotal',
                             'targetChargeTotal', 'date']]

transactions.rename(columns = {'id':'Transaction ID', 'projectId':'Project ID',
                               'transactionTypeId': 'Transaction Type',
                               'rateType': 'Rate Type', 'status': 'Status',
                               'stageId': 'Stage ID', 'date':'Date',
                               'invoiceValueTotal': 'Invoice Value Total',
                               'actualCostTotal':'Actual Cost Total',
                               'targetChargeTotal':'Target Charge Total',
                               'valueTotal':'Value Total',
                               'units': 'Units'}, inplace = True)

transactions = transactions[(transactions['Status'] == 'Invoiced') | (transactions['Status'] == 'Written off')]
transactions['Transaction Type'].replace([100, 200, 300, 400, 500, 700, 750, 800],
                                         ['Time', 'Cash', 'Travel', 'Office', 'Bill', 'Balance', 'Unearned', 'Invoice Custom'], inplace=True)
transactions['Date'] = pd.to_datetime(transactions['Date'])
transactions.columns = transactions.columns.str.replace(' ', '_')

transactions.head(1)

,Transaction_ID,Project_ID,Stage_ID,Transaction_Type,Rate_Type,Status,Units,Value_Total,Invoice_Value_Total,Actual_Cost_Total,Target_Charge_Total,Date
9,45264071,375028,1427835,Time,Staff,Invoiced,1.5,450.0,491.33,360.735,360.735,2021-08-26


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1083346 entries, 9 to 1624369
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Transaction_ID       1083346 non-null  int64         
 1   Project_ID           1083346 non-null  int64         
 2   Stage_ID             1083346 non-null  int64         
 3   Transaction_Type     1083346 non-null  object        
 4   Rate_Type            1083346 non-null  object        
 5   Status               1083346 non-null  object        
 6   Units                1083346 non-null  float64       
 7   Value_Total          1083346 non-null  float64       
 8   Invoice_Value_Total  1083346 non-null  float64       
 9   Actual_Cost_Total    1083346 non-null  float64       
 10  Target_Charge_Total  1083346 non-null  float64       
 11  Date                 1083346 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(3), object(3)
me

In [ ]:
# URL: https://stackoverflow.com/questions/21295334/find-length-of-longest-string-in-pandas-dataframe-column
for column in transactions:
    if transactions[column].dtype == 'object':
        print('Max length of column %s: %s\n' %  (column, transactions[column].map(len).max()))

Max length of column Transaction_Type: 7

Max length of column Rate_Type: 14

Max length of column Status: 11



**4 features to engineer:**
* Perc_of_Subcontractors (move to 'Projects' table)
* Is_Front_Loaded (move to 'Projects' table)
* Avg_Profit (move to 'Projects' table) - average profit margin per project
* Avg_Rec (move to 'Projects' table) - average financial recoverability per project

**2 features to update:**
* Project_Start_Date, Project_End_Date for projects with absent dates in the 'projects' table.

**Table alterations:**
* FK Time_Profile (links 'wga.projects' table on 'Project_ID')

Perc_of_Subcontractors = 
* total units of subcontractors divided by
* sum of units where transaction type is 'bill' or 'time'

* 'Time' = Company's employees
* 'Bill' = Hired subcontrators
* Time + Bill = total human capital on project in hours

In [ ]:
# Perc_of_Subcontractors
subs = transactions[['Project_ID', 'Units', 'Rate_Type']]
subs = subs[(subs['Rate_Type'] == 'Subcontractor')]
subs.drop(columns = ['Rate_Type'], inplace = True)
subs = pd.DataFrame(subs.groupby(['Project_ID'])['Units'].count()).reset_index()
subs.rename(columns = {'Units': 'Sub_Hours_Per_Project'}, inplace = True)

total_hours = transactions[['Project_ID', 'Units', 'Transaction_Type']]
total_hours = total_hours[(total_hours['Transaction_Type'] == 'Time') | (total_hours['Transaction_Type'] == 'Bill')]
total_hours = pd.DataFrame(total_hours.groupby(['Project_ID'])['Units'].count()).reset_index()
total_hours.rename(columns = {'Units': 'Total_Hours_Per_Project'}, inplace = True)

df_1 = pd.merge(projects, subs, how='left', left_on='Project_ID', right_on='Project_ID')
df_2 = pd.merge(df_1, total_hours, how='left', left_on='Project_ID', right_on='Project_ID')
df_2['Sub_Hours_Per_Project'].fillna(0, inplace=True)
df_2['Total_Hours_Per_Project'].fillna(0, inplace=True)
df_2['Perc_of_Subcontractors'] = (df_2['Sub_Hours_Per_Project'] / df_2['Total_Hours_Per_Project']).round(decimals = 2)
df_2 = df_2[['Project_ID', 'Perc_of_Subcontractors']]

# Add the new feature to the 'Projects' table
projects = pd.merge(projects, df_2,  how='left', left_on='Project_ID', right_on='Project_ID')
len(projects)

9755

In [ ]:
# Is_Front_Loaded
project_dates = projects[['Project_ID', 'Project_Start_Date', 'Project_End_Date']]
df_3 = transactions[['Project_ID', 'Units', 'Date']]
df_3 = pd.merge(df_3, project_dates, how='left', left_on='Project_ID', right_on='Project_ID')

first_half = df_3[(df_3['Date']  < df_3['Project_Start_Date'] + (df_3['Project_End_Date'] - df_3['Project_Start_Date'])/2)] # finding mid-point between 2 dates
first_half = pd.DataFrame(first_half.groupby(['Project_ID'])['Units'].sum()).reset_index()
first_half.rename(columns = {'Units': '1st_Half_Units'}, inplace = True)

total_units = pd.DataFrame(df_3.groupby(['Project_ID'])['Units'].sum()).reset_index()
total_units.rename(columns = {'Units': 'Total_Effort_Units'}, inplace = True)

df_4 = pd.merge(total_units, first_half, how ='left', left_on='Project_ID', right_on='Project_ID')
df_4['Perc_Being_Front'] = df_4['1st_Half_Units']/df_4['Total_Effort_Units']
df_4['Is_Front_Loaded'] = (df_4['Perc_Being_Front']>=0.7)
df_4 = df_4[['Project_ID', 'Is_Front_Loaded']]

# Add the new feature to the 'Projects' table
projects = pd.merge(projects, df_4,  how='left', left_on='Project_ID', right_on='Project_ID')
len(projects)

9755

In [ ]:
# Recoverability, Profit_Measure
transactions = transactions[['Project_ID', 'Stage_ID', 'Value_Total', 'Invoice_Value_Total', 'Actual_Cost_Total', 'Target_Charge_Total', 'Date']]
transactions['Recoverability'] = transactions['Value_Total']/transactions['Target_Charge_Total']
infinites = transactions[(transactions['Recoverability'] == np.inf) | (transactions['Recoverability'] == -np.inf)]
#infinites['Target_Charge_Total'].sum() - shows that all cells in 'Target_Charge_Total' column are equal to 0, creating unwanted infinite values

transactions = transactions[(transactions['Target_Charge_Total'] != 0)]
#transactions['Recoverability'].min(), transactions['Recoverability'].max() - shows that there limits are real numbers, not infinite

transactions['Profit_Measure'] = transactions['Invoice_Value_Total']/transactions['Actual_Cost_Total']
#transactions['Profit_Measure'].min(), transactions['Profit_Measure'].max()  - shows that there limits are real numbers, not infinite
transactions = transactions[['Project_ID', 'Stage_ID', 'Date', 'Recoverability', 'Profit_Measure']]

# stage_transactions
df_5 = pd.DataFrame(transactions.groupby(['Project_ID', 'Stage_ID'])['Recoverability'].count()).reset_index()
df_5.rename(columns = {'Recoverability':'Count'}, inplace = True)
stage_transactions = pd.DataFrame(transactions.groupby(['Project_ID', 'Stage_ID'])['Recoverability', 'Profit_Measure'].sum()).reset_index()
stage_transactions = pd.merge(stage_transactions, df_5, how='left', on=['Project_ID', 'Stage_ID'])
stage_transactions['Avg_Rec'] = stage_transactions['Recoverability']/stage_transactions['Count']
stage_transactions['Avg_Profit'] = stage_transactions['Profit_Measure']/stage_transactions['Count']
stage_transactions = stage_transactions[['Project_ID', 'Stage_ID', 'Avg_Rec', 'Avg_Profit']]

# project_transactions
df_6 =  pd.DataFrame(transactions.groupby(['Project_ID'])['Recoverability'].count()).reset_index()
df_6.rename(columns = {'Recoverability':'Count'}, inplace = True)
project_transactions =  pd.DataFrame(transactions.groupby(['Project_ID'])['Recoverability', 'Profit_Measure'].sum()).reset_index()
project_transactions = pd.merge(project_transactions, df_6, how='left', on='Project_ID')
project_transactions['Avg_Rec'] = project_transactions['Recoverability']/project_transactions['Count']
project_transactions['Avg_Profit'] = project_transactions['Profit_Measure']/project_transactions['Count']
project_transactions = project_transactions[['Project_ID', 'Avg_Rec', 'Avg_Profit']]

# Add the 2 new features to the 'Projects' table
projects = pd.merge(projects, project_transactions, how='left', on='Project_ID')
len(projects)

C:\Users\MarfaPopova\AppData\Local\Temp\ipykernel_7796\116779817.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stage_transactions = pd.DataFrame(transactions.groupby(['Project_ID', 'Stage_ID'])['Recoverability', 'Profit_Measure'].sum()).reset_index()
C:\Users\MarfaPopova\AppData\Local\Temp\ipykernel_7796\116779817.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  project_transactions =  pd.DataFrame(transactions.groupby(['Project_ID'])['Recoverability', 'Profit_Measure'].sum()).reset_index()


9755

In [ ]:
print('Only ' + str(transactions['Project_ID'].nunique()) + ' projects have transaction recorded, meaning ' + str(len(projects) - transactions['Project_ID'].nunique()) + ' projects will be missing from transaction tables.')

Only 9128 projects have transaction recorded, meaning 627 projects will be missing from transaction tables.


In [ ]:
# Project_Start_Date, Project_End_Date

def nat_check(date):
    if type(date) == pd._libs.tslibs.nattype.NaTType:
        return True
    else:
        return False

min_dates =  pd.DataFrame(transactions.groupby(['Project_ID'])['Date'].min()).reset_index()
min_dates.rename(columns = {'Date':'Min_Date'}, inplace = True)
max_dates =  pd.DataFrame(transactions.groupby(['Project_ID'])['Date'].max()).reset_index()
max_dates.rename(columns = {'Date':'Max_Date'}, inplace = True)
all_dates = pd.merge(min_dates, max_dates, how='left', on='Project_ID')
projects = pd.merge(projects, all_dates, how='left', on='Project_ID')
len(projects)

9755

In [ ]:
projects['Project_Start_Date'] = projects['Project_Start_Date'].map(str)
projects['Project_End_Date'] = projects['Project_End_Date'].map(str)
projects['Due_Date'] = projects['Due_Date'].map(str)

projects.loc[projects['Project_Start_Date']=='NaT','Project_Start_Date']=projects['Min_Date']
projects.loc[projects['Project_End_Date']=='NaT','Project_End_Date']=projects['Max_Date']
projects.loc[projects['Due_Date']=='NaT','Due_Date']=projects['Max_Date']

projects['Project_Start_Date'] = pd.to_datetime(projects['Project_Start_Date'])
projects['Project_End_Date'] = pd.to_datetime(projects['Project_End_Date'])
projects['Due_Date'] = pd.to_datetime(projects['Due_Date'])

projects.drop(columns = ['Min_Date', 'Max_Date'], inplace = True)
len(projects)

9755

Now, let's go back to engieering date-dependent features with newly filled in values in the 'Projects' table.

In [ ]:
# Project_Duration_Weeks

projects['Project_Duration_Weeks'] = (projects['Project_End_Date'] - projects['Project_Start_Date']).astype('timedelta64[W]')
neg_weeks = projects[(projects['Project_Duration_Weeks'] < 0)]
neg_ids = neg_weeks['Project_ID'].tolist()
neg_weeks = (projects[projects['Project_ID'].isin(neg_ids)])
neg_weeks

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks
6048,376189,Australia,Complete,Health & Aged Care,Cameron Jackson,7612841,WGAVIC Pty Ltd,2019-09-23,1970-01-01,2019-09-24,Standard,1.0,0.0,0.0,0.0,False,2.075550,5.188875,-2595.0
9407,1311624,Australia,Complete,Traffic Transport (Bridges),Mario Saliba,7613137,WGAVIC Pty Ltd,2022-02-14,2021-12-21,2021-12-21,Non-standard,1.0,0.0,0.0,0.0,False,1.271695,0.000000,-8.0


In [ ]:
# Manually substitute those dates that don't make sense from transactions to created and updates dates from the opriginal API table.
projects.loc[projects['Project_ID'] == 376189, 'Project_Start_Date'] = pd.to_datetime('13/08/2019')
projects.loc[projects['Project_ID'] == 376189, 'Project_End_Date'] = pd.to_datetime('25/02/2020')

projects.loc[projects['Project_ID'] == 1311624, 'Project_Start_Date'] = pd.to_datetime('07/12/2021')
projects.loc[projects['Project_ID'] == 1311624, 'Project_End_Date'] = pd.to_datetime('24/01/2022')

In [ ]:
projects['Project_Duration_Weeks'] = (projects['Project_End_Date'] - projects['Project_Start_Date']).astype('timedelta64[W]')
neg_weeks = projects[(projects['Project_Duration_Weeks'] < 0)]
neg_ids = neg_weeks['Project_ID'].tolist()
neg_weeks = (projects[projects['Project_ID'].isin(neg_ids)])
neg_weeks

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks


In [ ]:
# Delivered_on_Time
    
Delivered_on_Time = {}

for due_date in projects['Due_Date']:
    for completed in projects['Project_End_Date']:
        if nat_check(due_date) == True:
            continue
        else:
            if due_date <= completed:
                Delivered_on_Time[due_date] = True
            else:
                Delivered_on_Time[due_date] = False

df_7 = pd.DataFrame([{'Due_Date': due_date, 'Delivered_on_Time': is_on_time} for (due_date, is_on_time) in Delivered_on_Time.items()])

projects = pd.merge(projects, df_7, how='left', on='Due_Date')
projects.head()

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks,Delivered_on_Time
0,367704,Australia,Complete,Ports & Marine,Mark Gilbert,7612852,Whyalla,2015-07-01,2015-08-01,2018-12-13,Standard,4.0,0.0,1.0,0.0,True,2.070711,0.991479,4.0,True
1,367705,Australia,Complete,Civic & Education Buildings,Geoff Wallbridge,7612773,WGASA Pty Ltd,2013-09-30,2018-04-23,2018-04-23,Standard,1.0,0.0,1.0,0.0,False,1.926671,0.000000,238.0,True
2,367706,Australia,Complete,Civic & Education Buildings,Loreto Taglienti,7612773,WGASA Pty Ltd,2017-02-06,2017-02-07,2017-02-07,Standard,1.0,0.0,1.0,0.0,False,1.936449,1.290966,0.0,True
3,367707,Australia,Complete,Commercial & Retail Buildings,Mark Gilbert,7612773,WGASA Pty Ltd,2014-12-22,2017-04-21,2017-04-21,Standard,3.0,0.0,1.0,0.0,False,2.053392,1.631426,121.0,True
4,367708,Australia,Complete,Civic & Education Buildings,Peter McBean,7612695,WGASA Pty Ltd,2017-08-21,2017-09-07,2017-09-07,Standard,7.0,0.0,1.0,0.0,False,1.082860,0.000000,2.0,True


In [ ]:
# Fully_In_Lockdown, Partially_In_Lockdown

Lockdown_Period = (pd.date_range(start='2020-03-16', end = '2020-11-21', freq='D')).to_series()

projects['Start_in_Lockdown'] = projects['Project_Start_Date'].isin([Lockdown_Period])
projects['End_in_Lockdown'] = projects['Project_End_Date'].isin([Lockdown_Period])
dates_prep = pd.concat([projects['Start_in_Lockdown'], projects['End_in_Lockdown']], axis = 1) #axis=1 specifies horizontal stacking

projects['Fully_In_Lockdown'] = pd.DataFrame(dates_prep.all(axis=1))
projects['Partially_In_Lockdown'] = pd.DataFrame(dates_prep.any(axis=1))

projects.drop(columns = ['Start_in_Lockdown', 'End_in_Lockdown'], inplace = True)

In [ ]:
# Suffered_Data_Loss

def data_loss_check(start_date, end_date):
    if start_date < pd.Timestamp('2018-07-15') and end_date < pd.Timestamp('2018-07-15'): #project started and ended before the acqusition
        return False
    elif start_date > pd.Timestamp('2018-07-15') and end_date > pd.Timestamp('2018-07-15'): #project started and ended after the acqusition
        return False
    elif start_date < pd.Timestamp('2018-07-15') and end_date > pd.Timestamp('2018-07-15'): #project started before the acqusition but ended after it
        return True

Data_Loss_Check = {}

for start_date in projects['Project_Start_Date']:
    for end_date in projects['Project_End_Date']:
        if (nat_check(start_date) or nat_check(end_date)) == True:
            continue
        else:
            if data_loss_check(start_date, end_date) == True:
                Data_Loss_Check[start_date.strftime(format = '%Y-%m-%d %H:%M:%S'), end_date.strftime(format = '%Y-%m-%d %H:%M:%S')] = True
            else:
                Data_Loss_Check[start_date.strftime(format = '%Y-%m-%d %H:%M:%S'), end_date.strftime(format = '%Y-%m-%d %H:%M:%S')] = False

In [ ]:
# URL: https://www.geeksforgeeks.org/python-program-to-convert-a-tuple-to-a-string/#:~:text=There%20are%20various%20approaches%20to%20convert%20a%20tuple,of%20the%20tuple%20and%20convert%20it%20into%20string.

def convertTuple(tup):
    string = ', '.join(tup)
    return string

Execution_Timeframe = Data_Loss_Check.copy()

for key in Execution_Timeframe.keys():
    Execution_Timeframe[key] = convertTuple(key)
    
# Changing keys of our final dictionary
Suffered_Data_Loss = dict(zip((Execution_Timeframe.values()), (Data_Loss_Check.values())))

In [ ]:
# Create a column in 'Projects' table to create merge on
projects['Execution_Timeframe'] = projects['Project_Start_Date'].map(str) + ', ' + projects['Project_End_Date'].map(str)
projects['Execution_Timeframe'][0]
type(projects['Execution_Timeframe'][0])

str

In [ ]:
# Check whether the two future columns will merge
list(Suffered_Data_Loss)[0] == projects['Execution_Timeframe'][0]

True

In [ ]:
df_8 = pd.DataFrame.from_dict(Suffered_Data_Loss, orient ='index')
df_8 = df_8.reset_index()
df_8.rename(columns = {'index':'Execution_Timeframe', 0:'Suffered_Data_Loss'}, inplace = True)

projects = pd.merge(projects, df_8, how='left', on='Execution_Timeframe')

In [ ]:
projects['Fully_In_Lockdown'].value_counts()

False    9755
Name: Fully_In_Lockdown, dtype: int64

In [ ]:
projects['Partially_In_Lockdown'].value_counts()

False    9755
Name: Partially_In_Lockdown, dtype: int64

In [ ]:
projects.drop(columns = ['Fully_In_Lockdown', 'Partially_In_Lockdown'], inplace = True)

### 2.3 <a class="anchor" id="2_3"></a> Stages

This section resulted in *aec.stages* table in RDS for Client X.

In [ ]:
# Read only valid projects' stages
stages = pd.read_csv('csv-files/wga_power_bi_stages.csv', encoding = 'ISO-8859-1', low_memory=False)
stages = (stages[stages['Project ID'].isin(valid_ids)])
stages = stages[(stages['Stage Type'] != 'Proposal')] # We only want professional fees
stages = stages[['Project ID', 'Stage ID', 'Stage Fee Type', 'Is Disbursement Stage',
                 'Stage Manager', 'Stage Discipline','Stage Start Date','Stage End Date']]

stages['Is Disbursement Stage'].replace(['No', 'Yes'], [False, True],inplace=True)
stages['Stage Start Date'] = pd.to_datetime(stages['Stage Start Date'])
stages['Stage End Date'] = pd.to_datetime(stages['Stage End Date'])
stages.columns = stages.columns.str.replace(' ', '_')

# Add profit and recoverability measures to the 'Stages' table
stages = pd.merge(stages, stage_transactions, how='left', left_on=['Project_ID', 'Stage_ID'], right_on = ['Project_ID', 'Stage_ID'])
stages

,Project_ID,Stage_ID,Stage_Fee_Type,Is_Disbursement_Stage,Stage_Manager,Stage_Discipline,Stage_Start_Date,Stage_End_Date,Avg_Rec,Avg_Profit
0,368035,1390483,Hourly rates,True,Rick Penfold,Design,NaT,NaT,NaN,NaN
1,368043,1390577,Hourly rates,True,Rick Penfold,Design,NaT,NaT,NaN,NaN
2,368046,1390619,Hourly rates,True,Rick Penfold,Design,NaT,NaT,NaN,NaN
3,368048,1390644,Hourly rates,True,Rick Penfold,Design,NaT,NaT,NaN,NaN
4,368049,1390688,Hourly rates,True,Rick Penfold,Design,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
60130,371765,1411541,Fixed fee,False,Rick Penfold,Design,NaT,NaT,1.455714,2.589816
60131,371798,1411772,Fixed fee,False,Rick Penfold,Design,NaT,NaT,1.607828,0.933321
60132,371820,1411847,Fixed fee,False,Rick Penfold,Design,NaT,NaT,1.580778,1.606574
60133,371976,1412474,Fixed fee,False,Rick Penfold,Design,NaT,NaT,1.328287,1.516198


In [ ]:
stages['Project_ID'].nunique()

9752

In [ ]:
stages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60135 entries, 0 to 60134
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Project_ID             60135 non-null  int64         
 1   Stage_ID               60135 non-null  int64         
 2   Stage_Fee_Type         60135 non-null  object        
 3   Is_Disbursement_Stage  60135 non-null  bool          
 4   Stage_Manager          60135 non-null  object        
 5   Stage_Discipline       60135 non-null  object        
 6   Stage_Start_Date       21481 non-null  datetime64[ns]
 7   Stage_End_Date         21422 non-null  datetime64[ns]
 8   Avg_Rec                38318 non-null  float64       
 9   Avg_Profit             38318 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 4.6+ MB


In [ ]:
for column in stages:
    if stages[column].dtype == 'object':
        print('Max length of column %s: %s\n' %  (column, stages[column].map(len).max()))

Max length of column Stage_Fee_Type: 19

Max length of column Stage_Manager: 23

Max length of column Stage_Discipline: 20



**1 feature to engineer:**
* Perc_of_Stages_with_Fixed_Fee

In [ ]:
# Perc_of_Stages_with_Fixed_Fee
df_9 = pd.DataFrame(stages.groupby(['Project_ID', 'Stage_Fee_Type'])['Stage_ID'].count()).reset_index()
df_10 = pd.DataFrame(stages.groupby(['Project_ID'])['Stage_Fee_Type'].count()).reset_index()
df_10.rename(columns = {'Stage_Fee_Type':'Total_Num_Stages'}, inplace = True)
df_10 = pd.merge(df_9, df_10, how='left', on='Project_ID')
df_10.rename(columns = {'Stage_ID':'Num_of_Stages_Per_Type'}, inplace = True)
df_11 = df_10[(df_10['Stage_Fee_Type'] == 'Fixed fee')]
df_11['Perc_of_Stages_with_Fixed_Fee'] = (df_11['Num_of_Stages_Per_Type'] / df_11['Total_Num_Stages']).round(decimals = 2)
df_11 = df_11[['Project_ID', 'Perc_of_Stages_with_Fixed_Fee']]
all_stages =  pd.merge(df_10, df_11,  how='left', on='Project_ID')
all_stages = all_stages.fillna(0)
test = pd.merge(stages, all_stages,  how='left', on='Project_ID')
test['Perc_of_Stages_with_Fixed_Fee'].isnull().sum()

C:\Users\MarfaPopova\AppData\Local\Temp\ipykernel_7796\858042782.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_11['Perc_of_Stages_with_Fixed_Fee'] = (df_11['Num_of_Stages_Per_Type'] / df_11['Total_Num_Stages']).round(decimals = 2)


0

In [ ]:
nulls = pd.DataFrame(test['Perc_of_Stages_with_Fixed_Fee'].isnull())
nulls.rename(columns = {'Perc_of_Stages_with_Fixed_Fee':'checker'}, inplace = True)
nulls = nulls.loc[nulls['checker'] == True]
indexes = list(nulls.index.values)
nulls = test.iloc[indexes]
nulls

,Project_ID,Stage_ID,Stage_Fee_Type_x,Is_Disbursement_Stage,Stage_Manager,Stage_Discipline,Stage_Start_Date,Stage_End_Date,Avg_Rec,Avg_Profit,Stage_Fee_Type_y,Num_of_Stages_Per_Type,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee


In [ ]:
# Since the projects with mising values don't have fixed fee at all, let's fill them with zero
test = test.fillna(0)
test['Perc_of_Stages_with_Fixed_Fee'].isnull().sum()

0

In [ ]:
test = test[['Project_ID', 'Total_Num_Stages', 'Perc_of_Stages_with_Fixed_Fee']]
test.drop_duplicates(inplace = True, ignore_index=True)
test

,Project_ID,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee
0,368035,5,0.00
1,368043,4,0.50
2,368046,10,0.00
3,368048,44,0.05
4,368049,2,0.00
...,...,...,...
9747,372172,2,0.00
9748,375796,1,1.00
9749,376064,1,1.00
9750,371175,2,0.00


In [ ]:
projects = pd.merge(projects, test,  how ='left', on ='Project_ID')
projects['Perc_of_Stages_with_Fixed_Fee'].isnull().sum()

3

In [ ]:
nulls = pd.DataFrame(projects['Perc_of_Stages_with_Fixed_Fee'].isnull())
nulls.rename(columns = {'Perc_of_Stages_with_Fixed_Fee':'checker'}, inplace = True)
nulls = nulls.loc[nulls['checker'] == True]
indexes = list(nulls.index.values)
nulls = projects.iloc[indexes]
nulls

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks,Delivered_on_Time,Execution_Timeframe,Suffered_Data_Loss,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee
1864,369819,Australia,Complete,Commercial & Retail Buildings,Nick Lelos,7612832,WGASA Pty Ltd,NaT,NaT,NaT,Standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"NaT, NaT",NaN,NaN,NaN
9656,1427650,Australia,Complete,"Justice, Corrections & Emergency Services",Peter James,7612877,WGASA Pty Ltd,2022-01-01,2022-12-31,NaT,Non-standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,52.0,NaN,"2022-01-01 00:00:00, 2022-12-31 00:00:00",False,NaN,NaN
9754,1567895,Australia,Active,Commercial & Retail Buildings,Nick Lelos,7612773,WGASA Pty Ltd,2022-05-04,2022-05-13,2022-04-05,Non-standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,"2022-05-04 00:00:00, 2022-05-13 00:00:00",False,NaN,NaN


In [ ]:
stages.loc[stages['Project_ID'] == 15907 | 59885 | 60137]

,Project_ID,Stage_ID,Stage_Fee_Type,Is_Disbursement_Stage,Stage_Manager,Stage_Discipline,Stage_Start_Date,Stage_End_Date,Avg_Rec,Avg_Profit


It turns out, 3 projects are not included in the 'stages' tables, so their stages attributes are not recorded.

In [ ]:
# Stage_Duration_Weeks
df_4 = pd.DataFrame(stages['Stage_Start_Date'].notnull() & stages['Stage_End_Date'].notnull())
df_4.rename(columns = {0:'checker'}, inplace = True)
df_4 = df_4.loc[df_4['checker'] == True]

stages = pd.merge(stages, df_4, left_index=True, right_index=True)
stages['Stage_Duration_Weeks'] = ((stages['Stage_End_Date'] - stages['Stage_Start_Date']).astype('timedelta64[W]'))
stages.drop(columns = 'checker', inplace = True)

### 2.4 <a class="anchor" id="2_4"></a> Data health

This section resulted in *aec.health* table in RDS for Client X.

In [ ]:
# Load only valid projects
health = pd.read_csv('csv-files/wga_power_bi_stages.csv', encoding = 'ISO-8859-1',  low_memory=False)
health = (health[health['Project ID'].isin(valid_ids)])

# Only leave columns that are relevant
health = health[['Project ID', 'Stage ID',
                 'Data Quality - Has Issues',
                 'Data Quality - Has Inactive Staff Resourced', 
                 'Health - % Duration Complete',
                 'Health - % Fee Used', 'Health - Stages With Alerts #']]

# Convert columns for unified style
health.rename(columns = {'Data Quality - Has Issues': 'DQ_Has_Issues',
                         'Data Quality - Has Inactive Staff Resourced':'DQ_Has_Inactive_Staff_Resourced',
                         'Health - % Duration Complete':'Health_Perc_Duration_Complete',
                         'Health - % Fee Used':'Health_Perc_Fee_Used',
                         'Health - Stages With Alerts #':'Alerts_Total_Per_Stage'}, inplace = True)

health['DQ_Has_Issues'].replace(['No', 'Yes'],[False, True],inplace=True)
health['DQ_Has_Inactive_Staff_Resourced'].replace(['No', 'Yes'],[False, True],inplace=True)
health.columns = health.columns.str.replace(' ', '_')

health.head(1)

,Project_ID,Stage_ID,DQ_Has_Issues,DQ_Has_Inactive_Staff_Resourced,Health_Perc_Duration_Complete,Health_Perc_Fee_Used,Alerts_Total_Per_Stage
0,368035,1390483,False,False,NaN,NaN,0


In [ ]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60245 entries, 0 to 76928
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Project_ID                       60245 non-null  int64  
 1   Stage_ID                         60245 non-null  int64  
 2   DQ_Has_Issues                    60245 non-null  bool   
 3   DQ_Has_Inactive_Staff_Resourced  60245 non-null  bool   
 4   Health_Perc_Duration_Complete    20858 non-null  object 
 5   Health_Perc_Fee_Used             36656 non-null  float64
 6   Alerts_Total_Per_Stage           60245 non-null  int64  
dtypes: bool(2), float64(1), int64(3), object(1)
memory usage: 2.9+ MB


In [ ]:
health['Health_Perc_Duration_Complete'].value_counts()

â            1215
1.104302477      93
55.33333333      90
3.63681592       76
1.350241546      66
               ... 
58.66666667       1
0.787321063       1
7.326315789       1
4.652892562       1
7.622222222       1
Name: Health_Perc_Duration_Complete, Length: 9349, dtype: int64

In [ ]:
(health.loc[health['Health_Perc_Duration_Complete'] == 'â']).head(1)

,Project_ID,Stage_ID,DQ_Has_Issues,DQ_Has_Inactive_Staff_Resourced,Health_Perc_Duration_Complete,Health_Perc_Fee_Used,Alerts_Total_Per_Stage
11280,1082353,4310559,False,False,â,NaN,0


I investigated 'Health - % Duration Complete' column for project ID 1082353 in the original CSV file. The 'â' symbol stands for the infinite sign in Microsoft Excel's encoding. Hence, calculation in DAX must have been incorrect, as percentage cannot be infinite, unless divided by 0. This is why I will switch all of the faulty values to nulls.

In [ ]:
health['Health_Perc_Duration_Complete'].replace({'â': None},inplace = True)
health['Health_Perc_Duration_Complete'] = health['Health_Perc_Duration_Complete'].astype('float')

In [ ]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60245 entries, 0 to 76928
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Project_ID                       60245 non-null  int64  
 1   Stage_ID                         60245 non-null  int64  
 2   DQ_Has_Issues                    60245 non-null  bool   
 3   DQ_Has_Inactive_Staff_Resourced  60245 non-null  bool   
 4   Health_Perc_Duration_Complete    19643 non-null  float64
 5   Health_Perc_Fee_Used             36656 non-null  float64
 6   Alerts_Total_Per_Stage           60245 non-null  int64  
dtypes: bool(2), float64(2), int64(3)
memory usage: 2.9 MB


In [ ]:
checker = health[health['Project_ID'].isin([368035]) == True]
checker = checker[['Project_ID', 'Stage_ID', 'Alerts_Total_Per_Stage']]
checker

,Project_ID,Stage_ID,Alerts_Total_Per_Stage
0,368035,1390483,0
26089,368035,1390482,1
76438,368035,1390484,1
76439,368035,1390485,0
76440,368035,1390486,0


**1 feature to engineer:**
* Total_Data_Issues

In [ ]:
# Alerts_Total_Per_Project
issues = health.groupby(['Project_ID'], sort=False).sum('Alerts_Total_Per_Stage').reset_index()
issues = issues[['Project_ID', 'Alerts_Total_Per_Stage']]
issues.rename(columns = {'Alerts_Total_Per_Stage':'Total_Data_Issues'}, inplace = True)
projects = pd.merge(projects, issues, how ='left', on='Project_ID')
projects.head(1)

,Project_ID,Country,Project_Status,Sector,Project_Director,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks,Delivered_on_Time,Execution_Timeframe,Suffered_Data_Loss,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee,Total_Data_Issues
0,367704,Australia,Complete,Ports & Marine,Mark Gilbert,7612852,Whyalla,2015-07-01,2015-08-01,2018-12-13,Standard,4.0,0.0,1.0,0.0,True,2.070711,0.991479,4.0,True,"2015-07-01 00:00:00, 2015-08-01 00:00:00",False,5.0,0.8,2.0


### 2.5 <a class="anchor" id="2_5"></a> Clients

This section resulted in *aec.clients* table in RDS for Client X.

Step 1: Cleaning all given data, from Synergy API and Power BI.

In [ ]:
# Step 1: Cleaning data from Synergy API.
api_clients = pd.read_csv('csv-files/wga_synergy_overnight_1_clients.csv')
api_clients.drop(columns = {'Client Name', 'Unnamed: 0', 'Contact Type', 'Organisation ID'}, inplace = True)
api_clients['Created Date'] = pd.to_datetime(api_clients['Created Date'])

# Step 2: Cleaning transformed PowerBI data from S2R Analytics.
pbi_clients = pd.read_csv('csv-files/wga_power_bi_clients.csv', encoding = 'ISO-8859-1')
pbi_clients = pbi_clients[['Client ID', 'Client Projects - Total No', 'Client Projects - First Project ID']]
pbi_clients.rename(columns = {'Client Projects - Total No': 'Client Projects Total No',
                              'Client Projects - First Project ID':'1st Project ID'}, inplace = True)

# Step 3: Merge the two 'Clients' tables together.
clients = pd.merge(api_clients, pbi_clients,  how='left', left_on='Client ID', right_on='Client ID')
clients.columns = clients.columns.str.replace(' ', '_')
clients.head(1)

,Client_ID,Created_Date,Client_Projects_Total_No,1st_Project_ID
0,10317738,2022-05-06,NaN,NaN


In [ ]:
clients['Client_ID'].nunique()

12306

**3 features to engineer:**
* Client_Duration_Months
* Client_Is_Repeated
* Client_Is_Recent

In [ ]:
# Client_Is_Repeated
clients['Client_Is_Repeated'] = clients['1st_Project_ID'].notnull()

# Client_Duration_Months
clients['Client_Duration_Months'] = datetime.now() - clients['Created_Date']
clients['Client_Duration_Months'] = (clients['Client_Duration_Months'].astype('timedelta64[M]'))
clients['Client_Duration_Months'].isnull().sum()

0

In [ ]:
# Client_Is_Recent

Client_Is_Recent = {}

for months in clients['Client_Duration_Months']:
    if months < 6:
        Client_Is_Recent[months] = True
    else:
        Client_Is_Recent[months] = False
         
df_12 = pd.DataFrame(
    [{'Client_Duration_Months': months, 'Client_Is_Recent': recent_status} for (months, recent_status) in Client_Is_Recent.items()])

clients = pd.merge(clients, df_12, how='left', on='Client_Duration_Months')
clients.head(1)

,Client_ID,Created_Date,Client_Projects_Total_No,1st_Project_ID,Client_Is_Repeated,Client_Duration_Months,Client_Is_Recent
0,10317738,2022-05-06,NaN,NaN,False,3.0,True


In [ ]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12306 entries, 0 to 12305
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Client_ID                 12306 non-null  int64         
 1   Created_Date              12306 non-null  datetime64[ns]
 2   Client_Projects_Total_No  3965 non-null   float64       
 3   1st_Project_ID            3965 non-null   float64       
 4   Client_Is_Repeated        12306 non-null  bool          
 5   Client_Duration_Months    12306 non-null  float64       
 6   Client_Is_Recent          12306 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float64(3), int64(1)
memory usage: 600.9 KB


### 2.6 <a class="anchor" id="2_6"></a> Staff

This section resulted in *aec.staff* table in RDS for Client X.

In [ ]:
active = pd.read_csv('csv-files/wga_synergy_overnight_1_staff.csv', parse_dates=['Employment Date'], dayfirst=True)
active = active[['Staff ID', 'Staff Name', 'Employment Date', 'Synergy Team']]
active['Employment Date'] = pd.to_datetime(active['Employment Date'])
active['Employment Months'] = ((datetime.now() - active['Employment Date']).astype('timedelta64[M]'))
active.columns = active.columns.str.replace(' ', '_')

leavers = pd.read_csv('csv-files/wga_synergy_overnight_1_staff_-_leavers.csv', parse_dates=['Employment Date', 'Termination Date'], dayfirst=True)
leavers = leavers[['Staff ID', 'Staff Name', 'Employment Date', 'Termination Date', 'Synergy Team']]
leavers['Employment Date'] = pd.to_datetime(leavers['Employment Date'])
leavers['Termination Date'] = pd.to_datetime(leavers['Termination Date'])
leavers['Employment Months'] = ((leavers['Termination Date'] - leavers['Employment Date']).astype('timedelta64[M]'))
leavers.columns = leavers.columns.str.replace(' ', '_')

staff = pd.concat([active, leavers], axis = 0, ignore_index = True) # axis=0 specifies vertical stacking
staff = staff[['Staff_ID', 'Staff_Name', 'Synergy_Team', 'Employment_Months', 'Employment_Date', 'Termination_Date']]
staff.head(1)

,Staff_ID,Staff_Name,Synergy_Team,Employment_Months,Employment_Date,Termination_Date
0,7612683,Mel Chittleborough,SA - Finance,262.0,2000-09-25,NaT


In [ ]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Staff_ID           825 non-null    int64         
 1   Staff_Name         825 non-null    object        
 2   Synergy_Team       825 non-null    object        
 3   Employment_Months  825 non-null    float64       
 4   Employment_Date    825 non-null    datetime64[ns]
 5   Termination_Date   360 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 38.8+ KB


In [ ]:
for column in staff:
    if staff[column].dtype == 'object':
        print('Max length of column %s: %s\n' %  (column, staff[column].map(len).max()))

Max length of column Staff_Name: 27

Max length of column Synergy_Team: 31



**4 features to engineer:**
* Director_Empl_Months (move it to 'Projects' table)
* Manager_Empl_Months (move it to 'Projects' table)
* Manager_Is_Recent (move it to 'Projects' table)
* Director_Is_Recent (move it to 'Projects' table)

In [ ]:
# Director_Is_Recent

directors = projects[['Project_ID', 'Project_Director', 'Project_Start_Date']]
directors = pd.merge(directors, staff, how='left', left_on='Project_Director', right_on='Staff_Name')
directors.rename(columns = {'Synergy_Team':'Director_Team', 'Employment_Months':'Director_Empl_Months'}, inplace = True)
directors['Months_Before_Project'] = (directors['Project_Start_Date'] - directors['Employment_Date']).astype('timedelta64[M]')
directors.drop(columns = ['Staff_Name', 'Project_Start_Date'], inplace = True)

Director_Is_Recent = {}

for months in directors['Months_Before_Project']:
    if np.isnan(months) == True:
        continue
    else:
        if months < 6:
            Director_Is_Recent[months] = True
        else:
            Director_Is_Recent[months] = False
        
df_13 = pd.DataFrame([{'Months_Before_Project': months, 'Director_Is_Recent': recent_status} for (months, recent_status) in Director_Is_Recent.items()])

directors = pd.merge(directors, df_13, how ='left', on='Months_Before_Project')
directors.drop(columns = ['Employment_Date', 'Termination_Date', 'Months_Before_Project'], inplace = True)
projects = pd.merge(projects, directors, how ='left', on=['Project_ID', 'Project_Director'])
projects.rename(columns = {'Staff_ID':'Director_ID'}, inplace = True)
projects.drop(columns = 'Project_Director', inplace = True)
len(projects)

9755

In [ ]:
# Manager_Is_Recent

managers = projects[['Project_ID', 'Manager_ID', 'Project_Start_Date']]
managers = pd.merge(managers, staff, how='left', left_on='Manager_ID', right_on='Staff_ID')
managers.rename(columns = {'Synergy_Team':'Manager_Team', 'Employment_Months':'Manager_Empl_Months'}, inplace = True)
managers['Months_Before_Project'] = (managers['Project_Start_Date'] - managers['Employment_Date']).astype('timedelta64[M]')
managers.drop(columns = ['Staff_ID', 'Staff_Name', 'Project_Start_Date'], inplace = True)  

Manager_Is_Recent = {}

for months in managers['Months_Before_Project']:
    if np.isnan(months) == True:
        continue
    else:
        if months < 6:
            Manager_Is_Recent[months] = True
        else:
            Manager_Is_Recent[months] = False
        
df_14 = pd.DataFrame([{'Months_Before_Project': months, 'Manager_Is_Recent': recent_status} for (months, recent_status) in Manager_Is_Recent.items()])

managers = pd.merge(managers, df_14, how ='left', on='Months_Before_Project')
managers = managers[['Project_ID', 'Manager_ID', 'Manager_Team', 'Manager_Empl_Months', 'Manager_Is_Recent']]

projects = pd.merge(projects, managers, how ='left', on=['Project_ID', 'Manager_ID'])
len(projects)

9755

### 2.7 <a class="anchor" id="2_7"></a> Clean-up

In [ ]:
projects

,Project_ID,Country,Project_Status,Sector,Manager_ID,Office,Project_Start_Date,Project_End_Date,Due_Date,Default_Rate_Group,Project_Size_Sort_Order,Is_Multi_Discipline_Project,Is_First_Client_Project,Perc_of_Subcontractors,Is_Front_Loaded,Avg_Rec,Avg_Profit,Project_Duration_Weeks,Delivered_on_Time,Execution_Timeframe,Suffered_Data_Loss,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee,Total_Data_Issues,Director_ID,Director_Team,Director_Empl_Months,Director_Is_Recent,Manager_Team,Manager_Empl_Months,Manager_Is_Recent
0,367704,Australia,Complete,Ports & Marine,7612852,Whyalla,2015-07-01,2015-08-01,2018-12-13,Standard,4.0,0.0,1.0,0.0,True,2.070711,0.991479,4.0,True,"2015-07-01 00:00:00, 2015-08-01 00:00:00",False,5.0,0.80,2.0,7612709.0,SA - Industrial,433.0,False,SA - Industrial,127.0,False
1,367705,Australia,Complete,Civic & Education Buildings,7612773,WGASA Pty Ltd,2013-09-30,2018-04-23,2018-04-23,Standard,1.0,0.0,1.0,0.0,False,1.926671,0.000000,238.0,True,"2013-09-30 00:00:00, 2018-04-23 00:00:00",False,4.0,0.00,2.0,7612747.0,SA - Buildings,433.0,False,SA - Buildings,319.0,False
2,367706,Australia,Complete,Civic & Education Buildings,7612773,WGASA Pty Ltd,2017-02-06,2017-02-07,2017-02-07,Standard,1.0,0.0,1.0,0.0,False,1.936449,1.290966,0.0,True,"2017-02-06 00:00:00, 2017-02-07 00:00:00",False,2.0,0.00,0.0,7612768.0,SA - Buildings,339.0,False,SA - Buildings,319.0,False
3,367707,Australia,Complete,Commercial & Retail Buildings,7612773,WGASA Pty Ltd,2014-12-22,2017-04-21,2017-04-21,Standard,3.0,0.0,1.0,0.0,False,2.053392,1.631426,121.0,True,"2014-12-22 00:00:00, 2017-04-21 00:00:00",False,4.0,0.25,1.0,7612709.0,SA - Industrial,433.0,False,SA - Buildings,319.0,False
4,367708,Australia,Complete,Civic & Education Buildings,7612695,WGASA Pty Ltd,2017-08-21,2017-09-07,2017-09-07,Standard,7.0,0.0,1.0,0.0,False,1.082860,0.000000,2.0,True,"2017-08-21 00:00:00, 2017-09-07 00:00:00",False,6.0,0.00,3.0,7612695.0,SA - Buildings,421.0,False,SA - Buildings,421.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,1524776,Australia,Active,Civic & Education Buildings,9501752,WGAVIC Pty Ltd,2022-05-03,2022-05-13,2022-04-29,Non-standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,True,"2022-05-03 00:00:00, 2022-05-13 00:00:00",False,3.0,0.67,1.0,7859152.0,VIC - Traffic,29.0,False,VIC - Traffic,12.0,False
9751,1529864,Australia,Complete,Civic & Education Buildings,7809410,WGAWA Pty Ltd,2022-04-29,2022-05-06,NaT,Non-standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,"2022-04-29 00:00:00, 2022-05-06 00:00:00",False,2.0,0.50,1.0,10018105.0,SA - Industrial,6.0,True,WA - Civil,29.0,False
9752,1538446,Australia,Active,Energy,7612860,WGAVIC Pty Ltd,2022-05-02,2022-05-27,2022-02-05,Non-standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,3.0,True,"2022-05-02 00:00:00, 2022-05-27 00:00:00",False,1.0,1.00,0.0,7612841.0,VIC - Support Services,288.0,False,VIC - Buildings,170.0,False
9753,1538447,Australia,Active,Energy,7612860,WGAVIC Pty Ltd,2022-05-02,2022-05-31,2022-02-05,Non-standard,1.0,0.0,0.0,NaN,NaN,NaN,NaN,4.0,True,"2022-05-02 00:00:00, 2022-05-31 00:00:00",False,1.0,1.00,1.0,7612841.0,VIC - Support Services,288.0,False,VIC - Buildings,170.0,False


In [ ]:
projects.columns

In [194]:
len(projects.columns)

31

In [195]:
# Drop columns unnecessary for analysis and rearrange
projects = projects[['Project_ID', 'Country', 'Office', 'Sector', 'Project_Size_Sort_Order', 'Project_Duration_Weeks',
                'Total_Num_Stages', 'Is_Multi_Discipline_Project', 'Is_First_Client_Project',
                'Default_Rate_Group', 'Perc_of_Stages_with_Fixed_Fee',
                'Manager_ID', 'Manager_Team', 'Manager_Is_Recent', 'Manager_Empl_Months',
                'Director_ID', 'Director_Team', 'Director_Is_Recent', 'Director_Empl_Months',
                'Perc_of_Subcontractors','Is_Front_Loaded', 'Delivered_on_Time', 'Suffered_Data_Loss',
                'Total_Data_Issues', 'Avg_Rec', 'Avg_Profit', 'Project_Start_Date', 'Project_End_Date']]

len(projects.columns)

28

Dropped 3 columns: Project_Status, Due_Date, Execution_Timeframe.

In [196]:
%who DataFrame

active	 all_dates	 all_projects	 all_stages	 api_clients	 api_projects	 checker	 clients	 custom_fields	 
dates_prep	 df_1	 df_10	 df_11	 df_12	 df_13	 df_14	 df_2	 df_3	 
df_4	 df_5	 df_6	 df_7	 df_8	 df_9	 directors	 external_projects	 first_half	 
health	 infinites	 issues	 leavers	 managers	 max_dates	 min_dates	 neg_weeks	 nulls	 
pbi_clients	 pbi_projects	 project_dates	 project_transactions	 projects	 staff	 stage_transactions	 stages	 subs	 
successful_projects	 test	 total_hours	 total_units	 transactions	 


In [197]:
# Release all dataframes from Python memory apart from final ones that go into the WGA schema
dfs = [active, all_dates, all_projects, all_stages, api_clients, api_projects, checker, custom_fields, dates_prep, df_1, df_10, df_11, df_12, df_13, df_14,
       df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, directors, external_projects, first_half, health, infinites, issues, leavers,
       managers, max_dates, min_dates, neg_weeks, nulls, pbi_clients, pbi_projects, project_dates, project_transactions, staff, stage_transactions, subs,
       successful_projects, test, total_hours, total_units]
del active, all_dates, all_projects, all_stages, api_clients, api_projects, checker, custom_fields, dates_prep, df_1, df_10, df_11, df_12, df_13, df_14, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, directors, external_projects, first_half, health, infinites, issues, leavers, managers,  max_dates, min_dates, neg_weeks, nulls, pbi_clients, pbi_projects, project_dates, project_transactions, staff, stage_transactions, subs, successful_projects, test, total_hours,total_units
del dfs

In [198]:
%who DataFrame

clients	 projects	 stages	 transactions	 


## Part 3: <a class="anchor" id="part3"></a> Data loading

### 3.1 <a class="anchor" id="3_1"></a> Database design and storage

* SQL code for the schema is available in ```ETL\database_mngt\schema.sql```.
* Check the code for attempts to load the data into the created MS Server database in```ETL\database_mngt\connecting_to_db.py``` and ``` ETL\data_loading.py ```.

### 3.2 <a class="anchor" id="3_2"></a> Conversion to flat file

In [199]:
# Flat file on stage level

stage_lvl = pd.merge(projects, clients, how='left', left_on='Project_ID', right_on='1st_Project_ID')
stage_lvl.drop(columns = ['1st_Project_ID', 'Created_Date'], inplace = True)
stage_lvl.rename(columns = {'Staff_ID':'Project_Manager'}, inplace = True)
stages_prep = stages[['Project_ID',  'Stage_ID', 'Is_Disbursement_Stage', 'Stage_Discipline', 'Stage_Fee_Type']]
stage_lvl = pd.merge(stage_lvl, stages_prep, how='left', on='Project_ID')
stage_lvl

,Project_ID,Country,Office,Sector,Project_Size_Sort_Order,Project_Duration_Weeks,Total_Num_Stages,Is_Multi_Discipline_Project,Is_First_Client_Project,Default_Rate_Group,Perc_of_Stages_with_Fixed_Fee,Manager_ID,Manager_Team,Manager_Is_Recent,Manager_Empl_Months,Director_ID,Director_Team,Director_Is_Recent,Director_Empl_Months,Perc_of_Subcontractors,Is_Front_Loaded,Delivered_on_Time,Suffered_Data_Loss,Total_Data_Issues,Avg_Rec,Avg_Profit,Project_Start_Date,Project_End_Date,Client_ID,Client_Projects_Total_No,Client_Is_Repeated,Client_Duration_Months,Client_Is_Recent,Stage_ID,Is_Disbursement_Stage,Stage_Discipline,Stage_Fee_Type
0,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.0,1.0,Standard,0.80,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,False,2.0,2.070711,0.991479,2015-07-01,2015-08-01,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee
1,367705,Australia,WGASA Pty Ltd,Civic & Education Buildings,1.0,238.0,4.0,0.0,1.0,Standard,0.00,7612773,SA - Buildings,False,319.0,7612747.0,SA - Buildings,False,433.0,0.0,False,True,False,2.0,1.926671,0.000000,2013-09-30,2018-04-23,7615975.0,1.0,True,140.0,False,NaN,NaN,NaN,NaN
2,367706,Australia,WGASA Pty Ltd,Civic & Education Buildings,1.0,0.0,2.0,0.0,1.0,Standard,0.00,7612773,SA - Buildings,False,319.0,7612768.0,SA - Buildings,False,339.0,0.0,False,True,False,0.0,1.936449,1.290966,2017-02-06,2017-02-07,7614333.0,3.0,True,152.0,False,NaN,NaN,NaN,NaN
3,367707,Australia,WGASA Pty Ltd,Commercial & Retail Buildings,3.0,121.0,4.0,0.0,1.0,Standard,0.25,7612773,SA - Buildings,False,319.0,7612709.0,SA - Industrial,False,433.0,0.0,False,True,False,1.0,2.053392,1.631426,2014-12-22,2017-04-21,7614635.0,1.0,True,152.0,False,NaN,NaN,NaN,NaN
4,367708,Australia,WGASA Pty Ltd,Civic & Education Buildings,7.0,2.0,6.0,0.0,1.0,Standard,0.00,7612695,SA - Buildings,False,421.0,7612695.0,SA - Buildings,False,421.0,0.0,False,True,False,3.0,1.082860,0.000000,2017-08-21,2017-09-07,7613925.0,13.0,True,152.0,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27139,1529864,Australia,WGAWA Pty Ltd,Civic & Education Buildings,1.0,1.0,2.0,0.0,0.0,Non-standard,0.50,7809410,WA - Civil,False,29.0,10018105.0,SA - Industrial,True,6.0,NaN,NaN,NaN,False,1.0,NaN,NaN,2022-04-29,2022-05-06,NaN,NaN,NaN,NaN,NaN,5998563.0,False,* NOT SET,Fixed fee
27140,1529864,Australia,WGAWA Pty Ltd,Civic & Education Buildings,1.0,1.0,2.0,0.0,0.0,Non-standard,0.50,7809410,WA - Civil,False,29.0,10018105.0,SA - Industrial,True,6.0,NaN,NaN,NaN,False,1.0,NaN,NaN,2022-04-29,2022-05-06,NaN,NaN,NaN,NaN,NaN,5998564.0,False,* NOT SET,Hourly rates
27141,1538446,Australia,WGAVIC Pty Ltd,Energy,1.0,3.0,1.0,0.0,0.0,Non-standard,1.00,7612860,VIC - Buildings,False,170.0,7612841.0,VIC - Support Services,False,288.0,NaN,NaN,True,False,0.0,NaN,NaN,2022-05-02,2022-05-27,NaN,NaN,NaN,NaN,NaN,6009400.0,False,* NOT SET,Fixed fee
27142,1538447,Australia,WGAVIC Pty Ltd,Energy,1.0,4.0,1.0,0.0,0.0,Non-standard,1.00,7612860,VIC - Buildings,False,170.0,7612841.0,VIC - Support Services,False,288.0,NaN,NaN,True,False,1.0,NaN,NaN,2022-05-02,2022-05-31,NaN,NaN,NaN,NaN,NaN,6009401.0,False,* NOT SET,Fixed fee


In [200]:
# Flat file on transaction level
transaction_lvl = pd.merge(stage_lvl, transactions, how='left', on=['Project_ID', 'Stage_ID'])
len(transaction_lvl.columns)

40

In [201]:
transaction_lvl = transaction_lvl[['Project_ID', 'Country', 'Office', 'Sector', 'Project_Size_Sort_Order', 'Project_Duration_Weeks',
'Total_Num_Stages',  'Perc_of_Stages_with_Fixed_Fee', 'Is_Multi_Discipline_Project', 'Is_First_Client_Project', 'Default_Rate_Group',
'Manager_ID', 'Manager_Team', 'Manager_Is_Recent', 'Manager_Empl_Months',
'Director_ID', 'Director_Team', 'Director_Is_Recent', 'Director_Empl_Months',
'Perc_of_Subcontractors','Is_Front_Loaded', 'Delivered_on_Time', 'Total_Data_Issues',
'Client_ID', 'Client_Projects_Total_No', 'Client_Is_Repeated', 'Client_Duration_Months', 'Client_Is_Recent',
'Stage_ID', 'Is_Disbursement_Stage', 'Stage_Discipline', 'Stage_Fee_Type', 'Recoverability', 'Profit_Measure']]
len(transaction_lvl.columns)

34

In [202]:
transaction_lvl

,Project_ID,Country,Office,Sector,Project_Size_Sort_Order,Project_Duration_Weeks,Total_Num_Stages,Perc_of_Stages_with_Fixed_Fee,Is_Multi_Discipline_Project,Is_First_Client_Project,Default_Rate_Group,Manager_ID,Manager_Team,Manager_Is_Recent,Manager_Empl_Months,Director_ID,Director_Team,Director_Is_Recent,Director_Empl_Months,Perc_of_Subcontractors,Is_Front_Loaded,Delivered_on_Time,Total_Data_Issues,Client_ID,Client_Projects_Total_No,Client_Is_Repeated,Client_Duration_Months,Client_Is_Recent,Stage_ID,Is_Disbursement_Stage,Stage_Discipline,Stage_Fee_Type,Recoverability,Profit_Measure
0,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.8,0.0,1.0,Standard,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,2.0,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee,2.424242,0.000000
1,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.8,0.0,1.0,Standard,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,2.0,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee,2.424242,0.000000
2,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.8,0.0,1.0,Standard,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,2.0,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee,2.424242,0.000000
3,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.8,0.0,1.0,Standard,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,2.0,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee,1.843658,1.284239
4,367704,Australia,Whyalla,Ports & Marine,4.0,4.0,5.0,0.8,0.0,1.0,Standard,7612852,SA - Industrial,False,127.0,7612709.0,SA - Industrial,False,433.0,0.0,True,True,2.0,7615441.0,6.0,True,148.0,False,1388262.0,False,Design,Fixed fee,1.728429,1.203816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505930,1529864,Australia,WGAWA Pty Ltd,Civic & Education Buildings,1.0,1.0,2.0,0.5,0.0,0.0,Non-standard,7809410,WA - Civil,False,29.0,10018105.0,SA - Industrial,True,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5998563.0,False,* NOT SET,Fixed fee,NaN,NaN
505931,1529864,Australia,WGAWA Pty Ltd,Civic & Education Buildings,1.0,1.0,2.0,0.5,0.0,0.0,Non-standard,7809410,WA - Civil,False,29.0,10018105.0,SA - Industrial,True,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5998564.0,False,* NOT SET,Hourly rates,NaN,NaN
505932,1538446,Australia,WGAVIC Pty Ltd,Energy,1.0,3.0,1.0,1.0,0.0,0.0,Non-standard,7612860,VIC - Buildings,False,170.0,7612841.0,VIC - Support Services,False,288.0,NaN,NaN,True,0.0,NaN,NaN,NaN,NaN,NaN,6009400.0,False,* NOT SET,Fixed fee,NaN,NaN
505933,1538447,Australia,WGAVIC Pty Ltd,Energy,1.0,4.0,1.0,1.0,0.0,0.0,Non-standard,7612860,VIC - Buildings,False,170.0,7612841.0,VIC - Support Services,False,288.0,NaN,NaN,True,1.0,NaN,NaN,NaN,NaN,NaN,6009401.0,False,* NOT SET,Fixed fee,NaN,NaN


Dropped 3 columns: Date, Recpverability, Profit_Measure.

In [203]:
# Save the final dataframes in CSV format
projects.to_csv('csv-files/project_lvl.csv', index=False)
stage_lvl.to_csv('csv-files/stage_lvl.csv', index=False)
transaction_lvl.to_csv('csv-files/transaction_lvl.csv', index=False)